# Regressão Logística

In [15]:
# Bibliotecas
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


import warnings

warnings.filterwarnings("ignore")

In [16]:
df = pd.read_csv(r'C:\Users\User\Desktop\Tech_Challenge_2\TechChallenger_2_Ibovespa\FinalDataset.csv')
df

,Unnamed: 0,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Vol._sqrt,Tendencia,...,Ultimo_Fechamento_lag_3_dias,Variacao_Pct_lag_3_dias,Volume_lag_3_dias,Amplitude_Dia_lag_3_dias,Variacao_Abertura_Fechamento_lag_3_dias,Ultimo_Fechamento_lag_5_dias,Variacao_Pct_lag_5_dias,Volume_lag_5_dias,Amplitude_Dia_lag_5_dias,Variacao_Abertura_Fechamento_lag_5_dias
0,702,2023-01-11,112.517,110.816,112.552,110.752,1.446000e+07,0.0153,3802.630668,Alta,...,108.964,0.0123,1.263000e+07,1.791,1.322,105.334,0.0112,1.445000e+07,1.712,1.167
1,701,2023-01-12,111.850,112.516,113.129,110.982,1.984000e+07,-0.0059,4454.211490,Baixa,...,109.130,0.0015,1.205000e+07,1.804,0.166,107.641,0.0219,1.551000e+07,2.410,2.305
2,700,2023-01-13,110.916,111.843,111.847,110.428,1.594000e+07,-0.0084,3992.492956,Baixa,...,110.817,0.0155,1.370000e+07,2.715,1.688,108.964,0.0123,1.263000e+07,1.791,1.322
3,699,2023-01-16,109.213,110.908,110.908,108.753,1.251000e+07,-0.0154,3536.947837,Baixa,...,112.517,0.0153,1.446000e+07,1.800,1.701,109.130,0.0015,1.205000e+07,1.804,0.166
4,698,2023-01-17,111.439,109.214,111.577,109.214,1.436000e+07,0.0204,3789.459064,Alta,...,111.850,-0.0059,1.984000e+07,2.147,-0.666,110.817,0.0155,1.370000e+07,2.715,1.688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,5,2025-10-24,146.172,145.721,147.240,145.721,6.490000e+09,0.0031,80560.536244,Alta,...,144.085,-0.0029,6.190000e+09,0.966,-0.424,143.399,0.0084,8.880000e+09,2.176,1.199
698,4,2025-10-27,146.969,146.174,147.977,146.174,6.450000e+09,0.0055,80311.892021,Alta,...,144.873,0.0055,7.020000e+09,1.009,0.779,144.509,0.0077,6.790000e+09,1.820,1.110
699,3,2025-10-28,147.429,146.970,147.811,146.575,7.110000e+09,0.0031,84320.815935,Alta,...,145.721,0.0059,6.600000e+09,1.477,0.840,144.085,-0.0029,6.190000e+09,0.966,-0.424
700,2,2025-10-29,148.633,147.430,149.067,147.430,8.000000e+09,0.0082,89442.719100,Alta,...,146.172,0.0031,6.490000e+09,1.519,0.451,144.873,0.0055,7.020000e+09,1.009,0.779


In [17]:
df = df.dropna().reset_index(drop=True)

# Mapear Target (Alta=1, Baixa=0)
# 'Target_Tendencia_Amanha' é o alvo correto para PREVER o futuro
df['Target_Bin'] = df['Target_Tendencia_Amanha'].map({'Alta': 1, 'Baixa': 0})

In [18]:
df.columns

Index(['Unnamed: 0', 'Data', 'Último', 'Abertura', 'Máxima', 'Mínima', 'Vol.',
       'Var%', 'Vol._sqrt', 'Tendencia', 'Target_Tendencia_Amanha',
       'Tendencia_Dia_Anterior', 'Amplitude_Dia',
       'Variacao_Abertura_Fechamento', 'Media_Movel_5_dias',
       'Media_Movel_20_dias', 'Media_Movel_50_dias', 'RSI', 'MACD',
       'MACD_Signal', 'BB_High', 'BB_Low', 'BB_Middle', 'ATR',
       'Diff_Normalizada_MM20', 'Variacao_Pct_Volume', 'Dia_da_Semana', 'Mes',
       'Ano', 'Ultimo_Fechamento_lag_1_dia', 'Variacao_Pct_lag_1_dia',
       'Volume_lag_1_dia', 'Amplitude_Dia_lag_1_dia',
       'Variacao_Abertura_Fechamento_lag_1_dia',
       'Ultimo_Fechamento_lag_2_dias', 'Variacao_Pct_lag_2_dias',
       'Volume_lag_2_dias', 'Amplitude_Dia_lag_2_dias',
       'Variacao_Abertura_Fechamento_lag_2_dias',
       'Ultimo_Fechamento_lag_3_dias', 'Variacao_Pct_lag_3_dias',
       'Volume_lag_3_dias', 'Amplitude_Dia_lag_3_dias',
       'Variacao_Abertura_Fechamento_lag_3_dias',
       'Ultimo

In [19]:
# 2. Seleção de Features (A Combinação Vencedora)
# Estas features capturam Momentum e a "Força" do movimento atual para projetar o amanhã
features = [
    'RSI', 
    'Variacao_Pct_Volume', 
    'Variacao_Pct_lag_1_dia', 
    'Variacao_Abertura_Fechamento',  # O "corpo" do candle é um forte preditor de continuidade
    'MACD_Signal'
]

X = df[features]
y = df['Target_Bin']

# 3. Divisão Treino/Teste (Manter lógica temporal do notebook: últimos 30 dias)
test_size = 30
X_train = X.iloc[:-test_size]
X_test = X.iloc[-test_size:]
y_train = y.iloc[:-test_size]
y_test = y.iloc[-test_size:]

# 4. Escalonamento (Crucial para Regressão Logística)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. Modelo
# Solver 'liblinear' costuma funcionar melhor em datasets menores/binários
model = LogisticRegression(random_state=42, solver='liblinear', C=1.0)
model.fit(X_train_scaled, y_train)

# 6. Avaliação
y_pred = model.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)

print(f"Acurácia do Modelo Preditivo (Amanhã): {acc:.2%}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

Acurácia do Modelo Preditivo (Amanhã): 80.00%

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.69      0.92      0.79        12
           1       0.93      0.72      0.81        18

    accuracy                           0.80        30
   macro avg       0.81      0.82      0.80        30
weighted avg       0.83      0.80      0.80        30

